## Module: VegET functions
- Created on: April 2020
- VERSION: numpy, gdal, python 3
- AUTHOR: Stefanie Kagone, Gabe Parrish, Tony Butzer

In [1]:
# Defining global variables for model function

import numpy as np

# start and end year and day of the year, i is a counter for looping
# start_year = 2013
# end_year = 2016
# DOY = 274
# i = 0

# defining low and high temperature values (in C°) to determine what is snow, sleet, rain
rf_low_thresh_temp = 0
rf_high_thresh_temp = 6
rf_value = 0.167
melt_factor = 0.06

# drainage coefficient = 1 - runoff coefficient
dc_coeff = 0.65  
rf_coeff = 1 - dc_coeff  

# constant values to adjust NDVI in the ET calculation
varA = 1.25
varB = 0.2 

# water adjustment
varC = 0.70

# reference ET bias correction 
varD = 0.85
varE = 1.25

print("Defined all constant variables for computation")

Defined all constant variables for computation


### Code to connect to data stored in a AWS Cloud Bucket

`import numpy as np  
from s3fs.core import S3FileSystem  
#file in the bucket  
bucket_file = 'ga-et-data/Cloud_Veg_ET/Data/TMAX/tmax_012_gw.tif.npy'  
#connecting to S3  
s3 = S3FileSystem()  
#Check out bucket file's shape/properties   
ARY = np.load(s3.open(bucket_file))   
ARY.shape`

In [2]:
# pip install --user s3fs

In [3]:
# Connecting model to soil data stored in a cloud bucket

# import numpy as nput
# from s3fs.core import S3FileSystem
# file in the bucket
##bucket_file = 'ga-et-data/Cloud_Veg_ET/Data/TMAX/tmax_012_gw.tif.npy'
#connecting to S3
##s3 = S3FileSystem()
#Check out bucket file's shape/properties
##ARY = np.load(s3.open(bucket_file))
##ARY.shape

# # load all soil rasters
# interception1 = 'ga-et-data/Cloud_Veg_ET/Data/SOILS/Intercept2016_gw.tif.npy'
# whc1 = 'ga-et-data/Cloud_Veg_ET/Data/SOILS/awc_nbliss_filled_gw.tif.npy'
# field_capacity1 = 'ga-et-data/Cloud_Veg_ET/Data/SOILS/fc_nbliss_filled_gw.tif.npy'
# saturation1 = 'ga-et-data/Cloud_Veg_ET/Data/SOILS/sat_nbliss_filled_gw.tif.npy'

# s3 = S3FileSystem()
# interception = np.load(s3.open(interception1))
# print(interception.shape)
# whc = np.load(s3.open(whc1))
# print(whc.shape)
# field_capacity = np.load(s3.open(field_capacity1))
# print(field_capacity.shape)
# saturation = np.load(s3.open(saturation1))
# print(saturation.shape)
# print(dir(saturation))

In [4]:
# help(saturation)

In [5]:
# Connecting model to data stored in a cloud bucket

# import numpy as np
# from s3fs.core import S3FileSystem

# year = 2016

# # create a loop to run the model for all datasets on the same day
# d = 185
# if d == 185:
#     print(d)
#     DOY = '{:03d}'.format(d)
#     print(f'today is {DOY}')
#     ndvif = 'ga-et-data/Cloud_Veg_ET/Data/NDVI/'+ str(year) + DOY + '.250_m_NDVI_gw.npy'
#     pptf = 'ga-et-data/Cloud_Veg_ET/Data/PPT/prec_'+ str(year) + DOY + '.tif_gw.npy'
#     petf ='ga-et-data/Cloud_Veg_ET/Data/ETO/medianETo'+ DOY + '.tif_gw.npy'
#     tavgf = 'ga-et-data/Cloud_Veg_ET/Data/TAVG/tavg_'+ DOY + '_gw.tif.npy'
#     tminf = 'ga-et-data/Cloud_Veg_ET/Data/TMIN/tmin_'+ DOY + '_gw.tif.npy'
#     tmaxf = 'ga-et-data/Cloud_Veg_ET/Data/TMAX/tmax_'+ DOY + '_gw.tif.npy'
    
#     s3 = S3FileSystem()
#     ndvi = np.load(s3.open(ndvif))
#     pet = np.load(s3.open(petf))
#     ppt = np.load(s3.open(pptf))
#     tavg = np.load(s3.open(tavgf))
#     tmin = np.load(s3.open(tminf))
#     tmax = np.load(s3.open(tmaxf))

#     # d += 1
    
# print(ndvi.shape, ppt.shape, tavg.shape, tmax.shape)

In [6]:
# ndvi.data

### Function: inital soil water

1. precipitation bias corrected (if required)
2. effective precip
3. intercepted precip
4. snow melt factor
5. RAIN
6. SWE
7. snowmelt
8. snow pack
9. intial soil water  = previous day soil water balance + RAIN + snow_melt

In [7]:
def output_rasters(arr, outdir, outname, sample_tiff=None, geo_dict=None):
    #print('this is the output for {}'.format(outname))
    #return
    
    # todo- Make better error handling for these scripts in general
    # geo_dict = {'transform' : '| 0.00, 0.00,-77.02 |', 'coord_sys' : 'EPSG:4326', 'h' : int(3124), 'w' : int(1938)}
    
    outpath = '{}{}'.format(outdir, outname)
    print('the outpath for file {} is {}'.format(outname, outpath))
    if sample_tiff != None:
        # get the geoinfo from sample tiff to output intermediate files
        ds = rasterio.open(sample_tiff)
        band1 = arr
        with rasterio.open(outpath, 'w', driver='GTiff', height=ds.height, width=ds.width, 
                           count=1, dtype='float64', crs=ds.crs, transform=ds.transform) as wrast:
            wrast.write(band1, indexes=1)
            
    elif geo_dict != None:
        # assume you're given a numpy array
        print('sample tiff output not yet implemented')
        band1 = arr
        with rasterio.open(outpath, 'w', driver='GTiff', height=geo_dict['h'], width=geo_dict['w'], 
                           count=1, dtype='float64', crs=geo_dict['crs'], transform=geo_dict['transform']) as wrast:
            wrast.write(band1, indexes=1)
    
    from_file = outpath
    bucket = 'dev-et-data'
    prefix_no_slash = 'v1DRB_outputs'
    s3cp_delete_local(from_file, bucket, prefix_no_slash) 
    
    
    

In [8]:
def soil_water(i, ppt, interception, tmin, tmax, tavg, melt_factor, rf_high_thresh_temp, rf_low_thresh_temp, 
               yest_swf=None, yest_snwpck=None):
    print('ppt min', np.min(ppt))
    print('ppt max', np.max(ppt))
    #ppt[ppt == -32767] = np.nan
    ppt[ppt <= -1] = np.nan
    ppt[ppt == 32767] = np.nan
    print('ppt min', np.min(ppt))
    print('ppt max', np.max(ppt))
    
    print('interception min', np.min(interception))
    #interception[interception == -2147483648] = np.nan
    #interception[interception <= -1.79769313486e308] = np.nan
    #interception[interception == -128.0] = np.nan
    #interception[interception == 90] = np.nan

    print('interception min', np.min(interception))
    
    #    -3.40282346639e38
    #    -3.4028235e+38
    print('tavg min', np.min(tavg))
    tavg[tavg <= -100] = np.nan
    print('tavg min', np.min(tavg))
    
    print('tmax min', np.min(tmax))
    tmax[tmax <= -100] = np.nan
    print('tmax min', np.min(tmax))
    
    tmin[tmin <= -100] = np.nan
    
    # Effective PPT 
    effppt = ppt * (1 - (interception / 100.0))
    # Intercepted PPT
    interception = ppt * (interception / 100.0)
    print('interception min', np.min(interception))
    
    # Snow pack
    # melt rate
    # Usage: Creates a melt rate value based on the relationship between max and min air temperatureto determine the snow melt and from there the snow pack extent
    melt_rate = melt_factor * ((tmax*tmax)-(tmax-tmin))
    # initialize the snow melt factor array
    snow_melt_fac = np.zeros(ppt.shape)
    # where(tavg <= rf_high_tresh_temp, 0, melt_rate)
    # where snow_melt is <= the high threshold temp set is equal to the melt factor rate
    snow_melt_fac[tavg <= rf_high_thresh_temp] = melt_rate[tavg <= rf_high_thresh_temp] 
    # otherwise make it 0  -- Operation happens in-place like i += x --
    snow_melt_fac[tavg > rf_high_thresh_temp] = 0
    
    if i == 0: # first day of model run to initalize and establish the soil water balance
               # and snow pack
        # rain fraction
        # Usage: Creates a fraction value based on average temperature that determines 
        # if the incoming precipiation is falling as rain, sleet, or snow.
        # if tavg <= 0, make it 0, else if tavg >= 6, make it 1, else (0.167*(tavg-6))
        rain_frac = np.zeros(ppt.shape)
        rain_frac[tavg <= rf_low_thresh_temp] = 0
        rain_frac[tavg >= rf_high_thresh_temp] = 1
        temp_diff_boolean = (tavg < rf_high_thresh_temp) | (tavg > rf_low_thresh_temp)
        rain_frac[temp_diff_boolean] = rf_value * (tavg[temp_diff_boolean] - rf_high_thresh_temp)
        
        RAIN = rain_frac * effppt
        SWE = np.zeros(ppt.shape)  # inital snowpack raster with only 0 values 
        snow_melt = SWE
        SNWpk = np.zeros(ppt.shape)  # inital snowpack raster with only 0 values
        SWi = (whc * 0.5) + effppt + snow_melt
        SWi[SWi < 0] = np.nan
        print('RAIN min', np.min(RAIN))
        print('SWi min', np.min(SWi))

        
        return SWi, SNWpk, RAIN, SWE, snow_melt
    
    else:
        # temporary
#         yest_snwpck = snwpck_arr_lst[i-1]
#         yest_swf =  swf_arr_lst[i-1]
                
        rain_frac = np.zeros(ppt.shape) # initialize the rain fraction array
        # Creates a fraction value based on average temperature that determines 
        # if the incoming precipiation is falling as rain, slied, or snow.
        # if tavg <= 0, make it 0, else if tavg >= 6, make it 1, else (0.167*(tavg-6))
        rain_frac[tavg <= rf_low_thresh_temp] = 0
        rain_frac[tavg >= rf_high_thresh_temp] = 1
        temp_diff_boolean = (tavg < rf_high_thresh_temp) | (tavg > rf_low_thresh_temp)
        rain_frac[temp_diff_boolean] = rf_value * (tavg[temp_diff_boolean] - rf_high_thresh_temp)

        RAIN = rain_frac * effppt
        SWE = (1 - rain_frac) * effppt
        
        # snow melt
        snow_melt = np.zeros(ppt.shape) 
        # snow_melt = if melt_rate <= (SWE + yesterday's snowpack), make it melt_rate, else (SWE + yesterday's snowpack)
        snow_melt_boolean = (melt_rate <= (SWE + yest_snwpck))
        snow_melt[snow_melt_boolean] = melt_rate[snow_melt_boolean]
        snow_melt[~snow_melt_boolean] = SWE[~snow_melt_boolean] + yest_snwpck[~snow_melt_boolean]
       
        SNWpk = np.zeros(ppt.shape)
        # today's snow pack = yesterdays's snow pack + snow water amount - snowmelt
        SNWpk1 = yest_snwpck + SWE - snow_melt
        # if SNWpk1 < 0, make it 0, else SNWpk1
        SNWpk[SNWpk1<0] = 0
        SNWpk[SNWpk1>=0] = SNWpk1[SNWpk1>=0]

        # intial soil water balance = yesterday's soil water balance + Rain + snow melt
        SWi = yest_swf + RAIN + snow_melt
        # SWi[SWi < 0] = np.nan
        
        print('RAIN min', np.min(RAIN))
        print('SWi min', np.min(SWi))
        print('SWE min', np.min(SWi))
    
        return SWi, SNWpk, RAIN, SWE, snow_melt

# yest_snwpck = np.zeros(ndvi.shape)
# yest_swf = np.zeros(ndvi.shape)
# When running the code in a separate function call you would hava a counter i and 
# refer to yesterdays snowpack and swf as below
# soil_water(a, b, c, d, snwpck_arr_lst[i-1])   
# i=1
# SWi = soil_water(i, ppt, interception, tmin, tmax, tavg, melt_factor, rf_high_thresh_temp, 
#                  rf_low_thresh_temp, yest_swf, yest_snwpck)

### Function: Surface runoff

1. surface runoff
2. deep drainage

In [9]:
def surface_runoff(SWi, saturation, field_capacity, whc, rf_coeff, geo_dict=None):
    # Deep Drainage
    # DDrain occurs if SWi > WHC, amount of DDrain is SAT <> WHC with a maximum DDrain of SAT - WHC
    
    saturation[saturation < 0] = np.nan
    field_capacity[field_capacity < 0] = np.nan
    whc[whc < 0] = np.nan
    
    sat_fc = saturation - field_capacity
    Rf1 = SWi - whc
    # if rf1 < 0, make it 0, else rf1
    Rf = np.zeros(SWi.shape)
    rf_boolean = (Rf1 >= 0)
    Rf[rf_boolean] = Rf1[rf_boolean]
    
    # Surface runoff
    # Runoff = Surface runoff + deep drainage
    SRf = np.zeros(SWi.shape)
    # SRf = if rf <= sat_fc, make it (rf * rf_coeff)(35% of the runoff value), else (rf - sat_fc) + (rf_coeff * sat_fc)
    SRf_boolean = (Rf <= sat_fc)
    SRf[SRf_boolean] = Rf[SRf_boolean] * rf_coeff
    SRf[~SRf_boolean] = (Rf[~SRf_boolean] - sat_fc[~SRf_boolean]) + rf_coeff * sat_fc[~SRf_boolean]
    
    DDrain = Rf - SRf
    
    print('Rf min', np.min(Rf))
    print('SRf min', np.min(SRf))
    print('DDrain min', np.min(DDrain))
  
    return DDrain, SRf

# DDrain, SRf = surface_runoff(SWi, saturation, field_capacity, whc, rf_coeff)
# print(DDrain, SRf)

### Function: Veg_ET

1. Evapotranspiration
2. final soil water (after PPT,ET,.. is accounted for)

In [10]:
def veg_et(varA, varB, varC, varD, varE, watermask, pet, ndvi, SWi):
    # etasw1A = np.zeros(ndvi.shape)
    etasw1 = np.zeros(ndvi.shape)
    etasw3 = np.zeros(ndvi.shape)
    etasw4 = np.zeros(ndvi.shape)
    etasw5 = np.zeros(ndvi.shape)
    etasw = np.zeros(ndvi.shape)
    SWf = np.zeros(ndvi.shape)

    
    etasw1A = (varA * ndvi + varB) * (pet * varD)
    etasw1B = (varA * ndvi) * (pet * varD)
    
    # etasw1 = if ndvi > 0.4, make it etasw1A, else etasw1B
    ndvi_boolean = (ndvi > 0.4)
    etasw1[ndvi_boolean] = etasw1A[ndvi_boolean]
    etasw1[~ndvi_boolean] = etasw1B[~ndvi_boolean]
    
    etasw2 = (SWi / (0.5 * whc)) * etasw1
    
    # etasw3 = if SWi > (0.5 * WHC), make it etasw1, else etasw2
    etasw3_boolean = (etasw3 > SWi)
    etasw3[etasw3_boolean] = etasw1[etasw3_boolean]
    etasw3[~etasw3_boolean] = etasw1[~etasw3_boolean]
    
    # etasw4 = if etasw3 > SWi, make it SWi, else etasw3
    etasw4_boolean = (etasw3 > SWi)
    etasw4[etasw4_boolean] = SWi[etasw4_boolean]
    etasw4[~etasw4_boolean] = etasw3[~etasw4_boolean]
    
    # etasw = if etasw4 > WHC, make it WHC, else etasw4
    etasw_boolean = (etasw4 > whc)
    etasw5[etasw_boolean] = whc[etasw_boolean]
    etasw5[~etasw_boolean] = etasw4[~etasw_boolean]
    
    # ETa of water bodies = 0.70 * (1.25*0.85*ETo)
    water_var = varC * varD * varE
    print(watermask.shape)
    
    print(pet.shape)
    etawater_boolean = (watermask == 1)
    print(etawater_boolean.shape)
    # put the final etasw values for no-water regions in the final array
    etasw[~etawater_boolean] = etasw5[~etawater_boolean]
    # if it is a water-region, etasw is a function of PET
    etasw[etawater_boolean] = pet[etawater_boolean] * water_var
    print(etasw.shape)
        
    SWf1 = SWi - etasw
    
    # SWf = if SWi > WHC, make it (WHC - etasw), else (if SWf1 < 0.0, make it 0.0, else SWf1)
    SWf_boolean = (SWi > whc)
    SWf_boolean2 = (SWf1 < 0.0)
    
    SWf[SWf_boolean] = whc[SWf_boolean] - etasw[SWf_boolean]
    SWf[SWf_boolean2] = 0
    SWf[~SWf_boolean2] = SWf1[~SWf_boolean2]
    
    return etasw, SWf, etasw5
    
# etasw, SWf = veg_et(varA, varB, pet, ndvi, SWi)    
# print(etasw, SWf)    

In [11]:
# === Delaware River basin study area geo info for geotiff outputs ===

# geo_dict = {'transform' : '| 0.00, 0.00,-77.02 |', 
#            'coord_sys' : 'EPSG:4326', 
#            'h' : int(3124), 'w' : int(1938)}

# print('model concept test')
# import os
# import numpy as np
# import rasterio
# from matplotlib import pyplot as plt

# # testfile = etasw

# ds = rasterio.open(r'geo_sample.tif')
# # print('transform', ds.transform)
# # print(ds.crs)
# # print(type(ds.height), ds.width)
# # band1 = ds.read(1)

# band1 = etasw
# with rasterio.open('test_tif.tif', 'w', driver='GTiff', height=ds.height, width=ds.width, 
#                        count=1, dtype='float64', crs=ds.crs, transform=ds.transform) as wrast:
#     wrast.write(band1, indexes=1)
# #     write(supervised, indexes=1)
    
    
    


In [12]:
# Bucket stuff
import os
import boto3

def s3cp_delete_local(from_file, bucket, prefix_no_slash):
    #bucket='dev-rhassan'
    #prefix_no_slash='tony'
    objecta='{}/{}'.format(prefix_no_slash,from_file)
    s3 = boto3.client('s3')
    with open(from_file, "rb") as f:
        s3.upload_fileobj(f, bucket, objecta)
    os.remove(from_file) 
    

In [13]:
# plt.imshow(band1, vmin=0, vmax=7, cmap='YlGn')
# plt.show()

In [14]:
# from rasterio.plot import show
# show(ds)

In [15]:
# test_ds = rasterio.open('test_tif.tif')
# show(test_ds, cmap='viridis', adjust='linear')

In [16]:
# test_band = test_ds.read(1)
# test_band[test_band < 0] = 0
# show(test_band)

In [17]:
print(
'''_ _            ___  ___  _  _ 
| | | ___  ___ | __>|_ _|| || |
| ' |/ ._>/ . || _>  | | |_/|_/
|__/ \___.\_. ||___> |_| <_><_>
           <___'                '''
)

import os
from datetime import datetime, timedelta, date
import numpy as np
from s3fs.core import S3FileSystem
import rasterio
import calendar



def return_statics():

    # load all soil rasters
    interception1 = 'ga-et-data/Cloud_Veg_ET/Data/SOILS/Intercept2016_gw.tif.npy'
    whc1 = 'ga-et-data/Cloud_Veg_ET/Data/SOILS/awc_nbliss_filled_gw.tif.npy'
    field_capacity1 = 'ga-et-data/Cloud_Veg_ET/Data/SOILS/fc_nbliss_filled_gw.tif.npy'
    saturation1 = 'ga-et-data/Cloud_Veg_ET/Data/SOILS/sat_nbliss_filled_gw.tif.npy'
    s3 = S3FileSystem()
    interception = np.load('intercept_drb.npy')     #(s3.open(interception1))
    whc = np.load(s3.open(whc1))
    field_capacity = np.load(s3.open(field_capacity1))
    saturation = np.load(s3.open(saturation1))
    watermask= np.load('us_water_mask.npy')
    
    return interception, whc, field_capacity, saturation, watermask


       
        
        
# create a loop to run the model for all datasets on the same day
def run_water_bal(i, today, interception, whc, field_capacity, saturation,
                  rf_coeff, varA, varB, varC, varD, varE, watermask, outdir, 
                  yest_snwpck=None, yest_swf=None, 
                  geo_dict=None, sample_tiff=None, 
                  daily_mode=True):

    year = today.year
    print(year, today.month, today.day)
    DOY = '{:03d}'.format(today.timetuple().tm_yday)
    print(f'today is {DOY}')
    ndvif = 'ga-et-data/Cloud_Veg_ET/Data/NDVI/'+ str(year) + DOY + '.250_m_NDVI_gw.npy'
    pptf = 'ga-et-data/Cloud_Veg_ET/Data/PPT/prec_'+ str(year) + DOY + '.tif_gw.npy'
    petf ='ga-et-data/Cloud_Veg_ET/Data/ETO/medianETo'+ DOY + '.tif_gw.npy'
    tavgf = 'ga-et-data/Cloud_Veg_ET/Data/TAVG/tavg_'+ DOY + '_gw.tif.npy'
    tminf = 'ga-et-data/Cloud_Veg_ET/Data/TMIN/tmin_'+ DOY + '_gw.tif.npy'
    tmaxf = 'ga-et-data/Cloud_Veg_ET/Data/TMAX/tmax_'+ DOY + '_gw.tif.npy'
    
    s3 = S3FileSystem()
    ndvi = np.load(s3.open(ndvif))
    pet = np.load(s3.open(petf))
    ppt = np.load(s3.open(pptf))
    tavg = np.load(s3.open(tavgf))
    tmin = np.load(s3.open(tminf))
    tmax = np.load(s3.open(tmaxf))

#     ndvi = np.ones((3124,1938))
#     pet = np.ones((3124,1938))
#     ppt = np.ones((3124,1938))
#     tavg = np.ones((3124,1938))
#     tmax = np.ones((3124,1938))
#     tmin = np.ones((3124,1938))

    
    # ====== Call the functions ======
    # output SWi and SNWpk
    SWi, SNWpk, RAIN, SWE, snow_melt = soil_water(i, ppt, interception, tmin, tmax, tavg, 
                            melt_factor, rf_high_thresh_temp, rf_low_thresh_temp, 
                            yest_swf, yest_snwpck)
    SWiout = f'model_outputs/swi_{year}{DOY}.tif'
    SNWpkout = f'model_outputs/snwpk_{year}{DOY}.tif'
    RAINout = f'model_outputs/rain_{year}{DOY}.tif'
    SWEout = f'model_outputs/swe_{year}{DOY}.tif'
    snow_meltout = f'model_outputs/snowmelt_{year}{DOY}.tif'
    
    if daily_mode:
        output_rasters(SWi, outdir,outname=SWiout, sample_tiff=sample_tiff, geo_dict=geo_dict)
        output_rasters(SNWpk, outdir,outname=SNWpkout, sample_tiff=sample_tiff, geo_dict=geo_dict)
        output_rasters(RAIN, outdir,outname=RAINout, sample_tiff=sample_tiff, geo_dict=geo_dict)
        output_rasters(SWE, outdir,outname=SWEout, sample_tiff=sample_tiff, geo_dict=geo_dict)
        output_rasters(snow_melt, outdir,outname=snow_meltout, sample_tiff=sample_tiff, geo_dict=geo_dict)
    
    
    # output DDRAIN and SRf
    DDrain, SRf =surface_runoff(SWi, saturation, field_capacity, whc, rf_coeff, geo_dict=geo_dict)
    DDrainout = f'model_outputs/dd_{year}{DOY}.tif'
    SRfout = f'model_outputs/srf_{year}{DOY}.tif'
    if daily_mode:
        output_rasters(DDrain, outdir,outname=DDrainout, sample_tiff=sample_tiff, geo_dict=geo_dict)
        output_rasters(SRf, outdir,outname=SRfout, sample_tiff=sample_tiff, geo_dict=geo_dict)
        
    # output eta and SWf
    etasw, SWf, etasw5 = veg_et(varA, varB, varC, varD, varE, watermask, pet, ndvi, SWi)
    etaswout = f'model_outputs/etasw_{year}{DOY}.tif'
    SWfout = f'model_outputs/swf_{year}{DOY}.tif'
    etasw5out = f'model_outputs/etasw5_{year}{DOY}.tif'
    if daily_mode:
        output_rasters(etasw, outdir,outname=etaswout, sample_tiff=sample_tiff, geo_dict=geo_dict)
        output_rasters(SWf, outdir,outname=SWfout, sample_tiff=sample_tiff, geo_dict=geo_dict)
        output_rasters(etasw5, outdir,outname=etasw5out, sample_tiff=sample_tiff, geo_dict=geo_dict)
      
    return SWf, SNWpk, etasw, DDrain, SRf
   

def end_of_month_func(day, mon, year):

        #  calendar.monthrange return a tuple (weekday of first day of the 
        #  month, number  
        #  of days in month)
        last_day_of_month = calendar.monthrange(year, mon)[1]
        # here i check if date is last day of month
        if date(year, mon, day) == date(year, mon, last_day_of_month):
            return True
        
        return False


    
def run_veg_et(start_year, end_year, start_day, end_day, 
               interception, whc, field_capacity, saturation,
               rf_coeff, varA, varB, varC, varD, varE, watermask, 
               geo_dict=None, sample_tiff=None, outdir=None, accumulate_mode=None):
    
    start_dt = datetime.strptime("{}-{:03d}".format(start_year, start_day), '%Y-%j')
    print(start_dt)
    end_dt = datetime.strptime("{}-{:03d}".format(end_year, end_day), '%Y-%j')
    print(end_dt)
    time_interval = end_dt - start_dt
    num_days = time_interval.days
    print(num_days)
    
    # initially set output_yearly_arrya and output_monhly array to False and you will change 
    #them later depending on what is in the accumulate_mode list
    output_monthly_arr=False
    output_yearly_arr=False
    # sted daily. It is false if not included by default.
    if 'daily' not in accumulate_mode:
        output_daily_arr = False
    else: 
        output_daily_arr = True
    
   
    sample_shape = None
    # set monthly and yearly cumulative arrays:
    if sample_tiff != None:
        sampleds = rasterio.open(sample_tiff)
        sampleband = sampleds.read(1)
        sample_shape = sampleband.shape
        # A total of six output arrays must be instantiated in case accumulate_mode != None
        # monthly
        et_month_cum_arr = np.zeros(sample_shape)
        dd_month_cum_arr = np.zeros(sample_shape)
        srf_month_cum_arr = np.zeros(sample_shape)
        # yearly
        et_yearly_cum_arr = np.zeros(sample_shape)
        dd_yearly_cum_arr = np.zeros(sample_shape)
        srf_yearly_cum_arr = np.zeros(sample_shape)

    elif geo_dict != None:
        # todo - for steffi l8er? using (width, height)
        pass
    
    # the soil water fraction and snowpack are none to start out.
    changing_swf = None
    changing_snwpck = None
    for i in range(num_days + 1):
        # so what day is it
        today = start_dt + timedelta(days=i)
        if i == 0:
            swf, snwpck, etasw, DDrain, SRf = run_water_bal(i, today, interception, whc, field_capacity, saturation,
                                        rf_coeff, varA, varB, varC, varD, varE, watermask, outdir=outdir, 
                                        yest_snwpck=None, yest_swf=None, geo_dict=geo_dict, sample_tiff=sample_tiff, 
                                        daily_mode=output_daily_arr)
            changing_swf = swf
            changing_snwpck = snwpck
            
        else:
            
            # see if today is a day that we need to output a monthly raster
            if 'monthly' in accumulate_mode:
                d = today.day
                mo = today.month
                yr = today.year
                output_monthly_arr = end_of_month_func(d, mo, yr)
               
            
            if 'yearly' in accumulate_mode:
                # todo - deal with Water Year mode later
                
                # this function does calendar years
                d = today.day
                mo = today.month
                
                if d == 31 and mo == 12:
                    output_yearly_arr = True
            else:
                output_yearly_arr = False
                    
                      
            print('output monthly is {} and output yearly is {}'.format(output_monthly_arr, output_yearly_arr))
            
            swf, snwpck, etasw, DDrain, SRf = run_water_bal(i, today, interception, whc, field_capacity, saturation,
                                                            rf_coeff, varA, varB, varC, varD, varE, watermask, 
                                                            outdir=outdir, yest_snwpck=changing_snwpck,
                                                            yest_swf=changing_swf, geo_dict=geo_dict, 
                                                            sample_tiff=sample_tiff, daily_mode=output_daily_arr)
            # monthly
            et_month_cum_arr += etasw
            dd_month_cum_arr += DDrain
            srf_month_cum_arr += SRf
            # yearly
            et_yearly_cum_arr += etasw
            dd_yearly_cum_arr += DDrain
            srf_yearly_cum_arr += SRf
            
            if output_monthly_arr:
                # function to create monthly output rasters for each variable
                output_rasters(et_month_cum_arr, outdir, 'model_outputs/etasw_{}{:02d}.tif'.format(today.year, today.month), sample_tiff=sample_tiff, geo_dict=geo_dict)
                output_rasters(dd_month_cum_arr, outdir, 'model_outputs/dd_{}{:02d}.tif'.format(today.year, today.month), sample_tiff=sample_tiff, geo_dict=geo_dict)
                output_rasters(srf_month_cum_arr, outdir, 'model_outputs/srf_{}{:02d}.tif'.format(today.year, today.month), sample_tiff=sample_tiff, geo_dict=geo_dict)
                
                # zero-out arrays to start the next month over.
                et_month_cum_arr = np.zeros(sample_shape)
                dd_month_cum_arr = np.zeros(sample_shape)
                srf_month_cum_arr = np.zeros(sample_shape)
                output_monthly_arr = False
                
            if output_yearly_arr:
                # function to create yearly output rasters for each variables
                output_rasters(et_yearly_cum_arr, outdir, 'model_outputs/etasw_{}.tif'.format(today.year), sample_tiff=sample_tiff, geo_dict=geo_dict)
                output_rasters(dd_yearly_cum_arr, outdir, 'model_outputs/dd_{}.tif'.format(today.year), sample_tiff=sample_tiff, geo_dict=geo_dict)
                output_rasters(srf_yearly_cum_arr, outdir, 'model_outputs/srf_{}.tif'.format(today.year), sample_tiff=sample_tiff, geo_dict=geo_dict)
                
                # zero-out arrays to start the next year over.
                et_yearly_cum_arr = np.zeros(sample_shape)
                dd_yearly_cum_arr = np.zeros(sample_shape)
                srf_yearly_cum_arr = np.zeros(sample_shape)
                output_yearly_arr = False
            
            changing_swf = swf
            changing_snwpck = snwpck
        
        print('-------------------------------')
            


_ _            ___  ___  _  _ 
| | | ___  ___ | __>|_ _|| || |
| ' |/ ._>/ . || _>  | | |_/|_/
|__/ \___.\_. ||___> |_| <_><_>
           <___'                


In [18]:
print(datetime.now())

start_year = 2013
end_year = 2016

start_day = 274
end_day = 366

sample_tiff = r'geo_sample.tif'

# todo - handle leap years, handle bad inputs, need to make command line inputs for all params? returns soils automatically for veg et

# Get soil rasters for model run
interception, whc, field_capacity, saturation, watermask = return_statics()

# run Veg ET model, Acceptable accumulate modes: 'monthly', 'yearly'
run_veg_et(start_year, end_year, start_day, end_day, 
           interception, whc, field_capacity, saturation,
           rf_coeff, varA, varB, varC, varD, varE, watermask,
           geo_dict=None, sample_tiff=sample_tiff, outdir='', 
           accumulate_mode=['daily', 'monthly', 'yearly'])

print(datetime.now())






2020-05-04 18:01:06.994801
2013-10-01 00:00:00
2016-12-31 00:00:00
1187
2013 10 1
today is 274
ppt min 0.0
ppt max 32767.0
ppt min nan
ppt max nan
interception min 0
interception min 0
tavg min -3.4028235e+38
tavg min nan
tmax min -3.4028235e+38
tmax min nan
interception min nan


/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:45: RuntimeWarning: invalid value encountered in less_equal
/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:47: RuntimeWarning: invalid value encountered in greater
/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:56: RuntimeWarning: invalid value encountered in less_equal
/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:57: RuntimeWarning: invalid value encountered in greater_equal
/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:58: RuntimeWarning: invalid value encountered in less
/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:58: RuntimeWarning: invalid value encountered in greater
/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:66: RuntimeWarning: invalid value encountered in less


RAIN min nan
SWi min nan
the outpath for file model_outputs/swi_2013274.tif is model_outputs/swi_2013274.tif
the outpath for file model_outputs/snwpk_2013274.tif is model_outputs/snwpk_2013274.tif
the outpath for file model_outputs/rain_2013274.tif is model_outputs/rain_2013274.tif
the outpath for file model_outputs/swe_2013274.tif is model_outputs/swe_2013274.tif
the outpath for file model_outputs/snowmelt_2013274.tif is model_outputs/snowmelt_2013274.tif


/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in greater_equal
  del sys.path[0]
/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in less_equal


Rf min 0.0
SRf min nan
DDrain min nan
the outpath for file model_outputs/dd_2013274.tif is model_outputs/dd_2013274.tif
the outpath for file model_outputs/srf_2013274.tif is model_outputs/srf_2013274.tif


...

the outpath for file model_outputs/snowmelt_2016336.tif is model_outputs/snowmelt_2016336.tif
Rf min 0.0
SRf min nan
DDrain min nan
the outpath for file model_outputs/dd_2016336.tif is model_outputs/dd_2016336.tif
the outpath for file model_outputs/srf_2016336.tif is model_outputs/srf_2016336.tif
(3124, 1938)
(3124, 1938)
(3124, 1938)
(3124, 1938)
the outpath for file model_outputs/etasw_2016336.tif is model_outputs/etasw_2016336.tif
the outpath for file model_outputs/swf_2016336.tif is model_outputs/swf_2016336.tif
the outpath for file model_outputs/etasw5_2016336.tif is model_outputs/etasw5_2016336.tif
-------------------------------
output monthly is False and output yearly is False
2016 12 2
today is 337
ppt min 0.0
ppt max 32767.0
ppt min nan
ppt max nan
interception min 0
interception min 0
tavg min -3.4028235e+38
tavg min nan
tmax min -3.4028235e+38
tmax min nan
interception min nan
RAIN min nan
SWi min nan
SWE min nan
the outpath for file model_outputs/swi_2016337.tif is model_

the outpath for file model_outputs/etasw5_2016342.tif is model_outputs/etasw5_2016342.tif
-------------------------------
output monthly is False and output yearly is False
2016 12 8
today is 343
ppt min 0.0
ppt max 32767.0
ppt min nan
ppt max nan
interception min 0
interception min 0
tavg min -3.4028235e+38
tavg min nan
tmax min -3.4028235e+38
tmax min nan
interception min nan
RAIN min nan
SWi min nan
SWE min nan
the outpath for file model_outputs/swi_2016343.tif is model_outputs/swi_2016343.tif
the outpath for file model_outputs/snwpk_2016343.tif is model_outputs/snwpk_2016343.tif
the outpath for file model_outputs/rain_2016343.tif is model_outputs/rain_2016343.tif
the outpath for file model_outputs/swe_2016343.tif is model_outputs/swe_2016343.tif
the outpath for file model_outputs/snowmelt_2016343.tif is model_outputs/snowmelt_2016343.tif
Rf min 0.0
SRf min nan
DDrain min nan
the outpath for file model_outputs/dd_2016343.tif is model_outputs/dd_2016343.tif
the outpath for file model

the outpath for file model_outputs/snwpk_2016349.tif is model_outputs/snwpk_2016349.tif
the outpath for file model_outputs/rain_2016349.tif is model_outputs/rain_2016349.tif
the outpath for file model_outputs/swe_2016349.tif is model_outputs/swe_2016349.tif
the outpath for file model_outputs/snowmelt_2016349.tif is model_outputs/snowmelt_2016349.tif
Rf min 0.0
SRf min nan
DDrain min nan
the outpath for file model_outputs/dd_2016349.tif is model_outputs/dd_2016349.tif
the outpath for file model_outputs/srf_2016349.tif is model_outputs/srf_2016349.tif
(3124, 1938)
(3124, 1938)
(3124, 1938)
(3124, 1938)
the outpath for file model_outputs/etasw_2016349.tif is model_outputs/etasw_2016349.tif
the outpath for file model_outputs/swf_2016349.tif is model_outputs/swf_2016349.tif
the outpath for file model_outputs/etasw5_2016349.tif is model_outputs/etasw5_2016349.tif
-------------------------------
output monthly is False and output yearly is False
2016 12 15
today is 350
ppt min 0.0
ppt max 327

(3124, 1938)
(3124, 1938)
(3124, 1938)
(3124, 1938)
the outpath for file model_outputs/etasw_2016355.tif is model_outputs/etasw_2016355.tif
the outpath for file model_outputs/swf_2016355.tif is model_outputs/swf_2016355.tif
the outpath for file model_outputs/etasw5_2016355.tif is model_outputs/etasw5_2016355.tif
-------------------------------
output monthly is False and output yearly is False
2016 12 21
today is 356
ppt min 0.0
ppt max 32767.0
ppt min nan
ppt max nan
interception min 0
interception min 0
tavg min -3.4028235e+38
tavg min nan
tmax min -3.4028235e+38
tmax min nan
interception min nan
RAIN min nan
SWi min nan
SWE min nan
the outpath for file model_outputs/swi_2016356.tif is model_outputs/swi_2016356.tif
the outpath for file model_outputs/snwpk_2016356.tif is model_outputs/snwpk_2016356.tif
the outpath for file model_outputs/rain_2016356.tif is model_outputs/rain_2016356.tif
the outpath for file model_outputs/swe_2016356.tif is model_outputs/swe_2016356.tif
the outpath for

RAIN min nan
SWi min nan
SWE min nan
the outpath for file model_outputs/swi_2016362.tif is model_outputs/swi_2016362.tif
the outpath for file model_outputs/snwpk_2016362.tif is model_outputs/snwpk_2016362.tif
the outpath for file model_outputs/rain_2016362.tif is model_outputs/rain_2016362.tif
the outpath for file model_outputs/swe_2016362.tif is model_outputs/swe_2016362.tif
the outpath for file model_outputs/snowmelt_2016362.tif is model_outputs/snowmelt_2016362.tif
Rf min 0.0
SRf min nan
DDrain min nan
the outpath for file model_outputs/dd_2016362.tif is model_outputs/dd_2016362.tif
the outpath for file model_outputs/srf_2016362.tif is model_outputs/srf_2016362.tif
(3124, 1938)
(3124, 1938)
(3124, 1938)
(3124, 1938)
the outpath for file model_outputs/etasw_2016362.tif is model_outputs/etasw_2016362.tif
the outpath for file model_outputs/swf_2016362.tif is model_outputs/swf_2016362.tif
the outpath for file model_outputs/etasw5_2016362.tif is model_outputs/etasw5_2016362.tif
---------